In [ ]:
import os
import sys
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()
import pyspark

In [ ]:
number_cores = 6
memory_gb = 16
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set("spark.driver.maxResultSize", "5g"))
sc = pyspark.SparkContext(conf=conf)

In [ ]:
!dir /users/trush/CSC496/Labs/Lab4

In [ ]:
!ls -lh /users/trush/CSC496/Labs/Lab4/data

# 1. Identify 100 users with highest number of ratings/reviews/followers.  You can decide/justify this ranking and decide the importance of ratings/reviews/followers. 

In [ ]:
# Step 1: Load the data using SQL Context

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
# Get SQL table
df_json = sqlContext.read.json("/users/trush/CSC496/Labs/Lab4/data/yelp_academic_dataset_user.json.gz")

In [ ]:
df_json.printSchema()

In [ ]:
df_json.registerTempTable("tbl_json")

In [ ]:
# Using Python
import json
def parseJSONReviews(x):
    res = json.loads(x)
    return (res['user_id'], int(res['review_count']), res['fans'])

In [ ]:
df_data = sc.textFile("/users/trush/CSC496/Labs/Lab4/data/yelp_academic_dataset_user.json.gz")
print(df_data.count())
df_data.take(1)

In [ ]:
df_highest = df_data.map(lambda x: parseJSONReviews(x)).takeOrdered(100, lambda x: -x[2])
df_highest

In [ ]:
# Using SQL
highest_fan = sqlContext.sql("SELECT user_id, review_count, fans FROM tbl_json ORDER BY fans DESC LIMIT 100")

In [ ]:
highest_fan.show()

# 2. Extract the reviews of these users and combine it with the business information.

### a. Are they continental, regional, or local eaters?
- Look at review data for business data. Look at business data for location.
- Variations in latitude and longitude of reviewed businesses (distance between furthest pair of restaurants, cluster into 3)

### b. Is there a preference in restaurant/food style of their reviews?
- Frequent itemsets
    - per user
        - set of items: categories
        - set of baskets: restaurants

### c. Can you infer the locations of these users?
- Locations of businesses most frequently visited (consider timestamp for users who go on trips to review restaurants)

#### How to prepare data to answer questions?
- List of users with highest fans (user_id, review_count, fans) - local
- Map users to reviews

In [ ]:
#raw_review_df.filter(lambda x: json.loads(x)["user_id"] in df_top_users)

In [ ]:
def extract_id(x):
    x_json = json.loads(x)
    return (x_json["user_id"])
tmp = df_data.take(1)
extract_id(tmp[0])

# 3. Identify one of your favorite restaurants that is available on Yelp. Search for all reviews and reviewers for this restaurants.  

### a. Is this restaurant frequented by non-local reviewers (how do you know)?
- Coach's

### b. What are the positive things about this restaurant (study higher-rated reviews)
- sentiment analysis

### c. What are the negative things about this restaurant (study lower-rated reviews)
- sentiment analysis

In [ ]:
raw_review_df = sc.textFile("/users/trush/CSC496/Labs/Lab4/data/yelp_academic_dataset_review.json.gz")
print(raw_review_df)
raw_review_df.take(1)

In [ ]:
user_reviews = df_user_review.map(lambda x: (x[1], 1))
user_reviews.take(10)

In [ ]:
df_user_reviews = raw_review_df.filter(lambda x: json.loads(x)["user_id"] in df_data)
print(df_user_reviews.count())
df_user_reviews.take(10)

In [ ]:
df_test = raw_review_df.take(1)
json.loads(df_test)["user_id"]

In [ ]:
sum = 0
for item in df_top_users:
    sum = sum + int(item[1])
sum